# PyCity Schools Analysis

- Your analysis here
  
---

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = len(pd.unique(school_data_complete["school_name"]))
school_count

15

In [3]:
# Calculate the total number of students
#39170
student_count = school_data_complete.student_name.count()
student_count

39170

In [4]:
# Pull unique schools and budget
# Calculate the total budget
#24649428
unique_school = school_data_complete[["school_name", "budget"]].copy()
unique_budget = unique_school.drop_duplicates(subset=["school_name"])
total_budget = unique_budget.budget.sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
#78.98537145774827
average_math_score = school_data_complete["math_score"].mean() 
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
#81.87784018381414
average_reading_score = school_data_complete["reading_score"].mean() 
average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
# (this was provided in the  PyCitySchools_starter.ipynb)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage =  passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
# (this was provided in the  PyCitySchools_starter.ipynb)
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary_data = {"Total Schools": [school_count], "Total Students": [student_count], "Total Budget": [total_budget], 
                        "Average Math Score": [average_math_score], "Average Reading Score": [average_reading_score],
                        "% Passing Math": [passing_math_percentage], "% Passing Reading": [passing_reading_percentage],
                        "% Overall Passing": [overall_passing_rate]}

# create data fram
district_summary = pd.DataFrame(district_summary_data)
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [13]:
# Use the code provided to select all of the school types (there was no code provided)
#use unique
school_types = school_data_complete["type"].unique()

In [45]:
# Calculate the total student count per school
per_school_counts = school_data_complete["school_name"].value_counts()

# Turn count into a df
per_school_counts_df = pd.DataFrame(per_school_counts)
# Reset Index and Rename columns
per_school_counts_df = per_school_counts_df.rename(columns={"school_name":"total_students"})
per_school_counts_df = per_school_counts_df.rename_axis("school_name").reset_index()
per_school_counts_df
#checked

,school_name,total_students
0,Bailey High School,4976
1,Johnson High School,4761
2,Hernandez High School,4635
3,Rodriguez High School,3999
4,Figueroa High School,2949
5,Huang High School,2917
6,Ford High School,2739
7,Wilson High School,2283
8,Cabrera High School,1858
9,Wright High School,1800


In [15]:
# Calculate the total school budget and per capita spending per school

merge_df = pd.merge(per_school_counts_df, unique_budget, on="school_name")

per_school_budget = merge_df.iloc[:,2]

#per_school_capita = 
merge_df["Per Student Budget"] = merge_df["budget"]/merge_df["total_students"]

In [17]:
# Calculate the average test scores per school
# Create a smaller data fram with testing info and group by school
school_scores= school_data_complete[["school_name", "reading_score", "math_score"]]
school_group = school_scores.groupby(["school_name"])

# Calculate averages for testing scores
school_ave_df = school_group.mean()

# Identify location for the respective testing scores for math and reading
per_school_math = school_ave_df.iloc[:,1]
per_school_reading = school_ave_df.iloc[:,0]
#checked  

In [49]:
# Calculate the number of students per school with math scores of 70 or higher
# Remove scores <70 for math; groupby school and count 
students_passing_math = school_data_complete.drop(school_data_complete[school_data_complete.math_score <70].index)
school_students_passing_math = students_passing_math.groupby(["school_name"]).count()
school_students_passing_math
#checked

# Turn count into a df
#school_students_passing_math_df = pd.DataFrame(school_students_passing_math)

# Reset Index and Rename columns
#school_students_passing_math_df = school_students_passing_math_df.rename(columns={"total_students":"passing_math"})

#school_students_passing_math_df.head()
#checked - NOT done, need to get rid of all but one column and rename and reset.  Merge all of the dataframes and then do the math.



,Student ID,student_name,gender,grade,reading_score,math_score,School ID,type,size,budget
school_name,,,,,,,,,,
Bailey High School,3318,3318,3318,3318,3318,3318,3318,3318,3318,3318
Cabrera High School,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749
Figueroa High School,1946,1946,1946,1946,1946,1946,1946,1946,1946,1946
Ford High School,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871
Griffin High School,1371,1371,1371,1371,1371,1371,1371,1371,1371,1371


In [37]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete.drop(school_data_complete[school_data_complete.reading_score <70].index)
school_students_passing_reading = school_students_passing_reading = students_passing_reading.groupby(["school_name"]).count()
school_students_passing_reading 
#checked






,Student ID,student_name,gender,grade,reading_score,math_score,School ID,type,size,budget
school_name,,,,,,,,,,
Bailey High School,4077,4077,4077,4077,4077,4077,4077,4077,4077,4077
Cabrera High School,1803,1803,1803,1803,1803,1803,1803,1803,1803,1803
Figueroa High School,2381,2381,2381,2381,2381,2381,2381,2381,2381,2381
Ford High School,2172,2172,2172,2172,2172,2172,2172,2172,2172,2172
Griffin High School,1426,1426,1426,1426,1426,1426,1426,1426,1426,1426
Hernandez High School,3748,3748,3748,3748,3748,3748,3748,3748,3748,3748
Holden High School,411,411,411,411,411,411,411,411,411,411
Huang High School,2372,2372,2372,2372,2372,2372,2372,2372,2372,2372
Johnson High School,3867,3867,3867,3867,3867,3867,3867,3867,3867,3867


In [28]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
#code provided and used directly
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
#school_students_passing_math_and_reading 
#checked

school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [30]:
# Use the provided code to calculate the passing rates
# Need to do this for each school - create a new dataframe to do the math and then do the math?

school_passing_rates_df = 

## Add column using arithmetic operation
# based on existing columns 
# df["Final_Fee"] = df["Fee"] - df["Discount"]
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

C:\Users\Aleid van der Zel\AppData\Local\Temp\ipykernel_18896\3548945202.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  per_school_passing_math = school_students_passing_math / per_school_counts * 100


ValueError: operands could not be broadcast together with shapes (15,2) (15,) 

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = 

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = 
top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = 
bottom_schools.head(5)

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = 
tenth_grader_math_scores = 
eleventh_grader_math_scores = 
twelfth_grader_math_scores = 

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = 

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = 
tenth_grader_reading_scores = 
eleventh_grader_reading_scores = 
twelfth_grader_reading_scores = 

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary